In [4]:
# 기본 작업 경로 설정

import os
notebook_path = os.path.abspath("../readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git


### https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md

In [2]:
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# 데이터셋 경로와 형식 설정
register_coco_instances("face_data_set", {}, "data/ssd_rcnn_face/annotations.json", "data/yolo_data/train/")
register_coco_instances("face_data_set_valid", {}, "data/ssd_rcnn_face/annotations_val.json", "data/yolo_data/val/")

# 메타데이터를 확인.
metadata = MetadataCatalog.get("face_data_set")
dataset_dicts = DatasetCatalog.get("face_data_set")

In [5]:
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.config import CfgNode as CN
import json
import os

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_dir=None):
        return COCOEvaluator(dataset_name, cfg, True, output_dir)
    
    def train(self):
        self.results = {"train": [], "test": []}
        super().train()
        
        # Add evaluation after each epoch
        for epoch in range(0, self.cfg.SOLVER.MAX_ITER + 1, self.cfg.TEST.EVAL_PERIOD):
            self.do_test(epoch)
            if epoch % self.cfg.SOLVER.CHECKPOINT_PERIOD == 0:
                self.checkpointer.save(f"model_{epoch}")

    def do_test(self, epoch):
        try:
            # Perform evaluation
            evaluator = self.build_evaluator(self.cfg, self.cfg.DATASETS.TEST[0], output_dir=self.cfg.OUTPUT_DIR)
            val_loader = build_detection_test_loader(self.cfg, self.cfg.DATASETS.TEST[0])
            results = inference_on_dataset(self.model, val_loader, evaluator)
            self.results["test"].append({"epoch": epoch, "results": results})
            print("Evaluation results:", results)
    
            # Save the results as JSON
            self.save_results_as_json()
        except Exception as e:
            print(f"Error during evaluation at epoch {epoch}: {e}")
    
    def save_results_as_json(self):
        try:
            output_file_path = os.path.join(self.cfg.OUTPUT_DIR, "train_test_results.json")
            print(f"Saving results to: {output_file_path}")
            with open(output_file_path, 'w') as f:
                json.dump(self.results, f, indent=2)
        except Exception as e:
            print(f"Error saving results as JSON: {e}")


# Set up configuration
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("face_data_set",)
cfg.DATASETS.TEST = ("face_data_set_valid",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 24020
cfg.INPUT.MIN_SIZE_TRAIN = 512
cfg.INPUT.MAX_SIZE_TRAIN = 512
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.RANDOM_ROTATION = 30
cfg.INPUT.CROP = CN({"ENABLED": True, "TYPE": "relative_range", "SIZE": [0.8, 0.8]})
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.OUTPUT_DIR = "./models/faster_rcnn_R_50_FPN_3x"
cfg.TEST.EVAL_PERIOD = 1000
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.WARMUP_ITERS = 500

setup_logger()
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()


[09/07 09:19:58 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=

[09/07 09:20:00 d2.engine.hooks]: Loading scheduler from state_dict ...
[09/07 09:20:00 d2.engine.train_loop]: Starting training from iteration 24000
[09/07 09:20:49 d2.utils.events]:  eta: 0:00:00  iter: 24019  total_loss: 0.1038  loss_cls: 0.01578  loss_box_reg: 0.08612  loss_rpn_cls: 3.186e-05  loss_rpn_loc: 0.001119    time: 2.3142  last_time: 2.1611  data_time: 1.4605  last_data_time: 1.2784   lr: 1.7106e-11  max_mem: 7035M
[09/07 09:20:49 d2.engine.hooks]: Overall training speed: 18 iterations in 0:00:41 (2.3142 s / it)
[09/07 09:20:49 d2.engine.hooks]: Total training time: 0:00:43 (0:00:01 on hooks)
[09/07 09:20:49 d2.data.datasets.coco]: Loaded 1199 images in COCO format from data/ssd_rcnn_face/annotations_val.json
[09/07 09:20:49 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(512, 512), max_size=512, sample_style='choice')]
[09/07 09:20:49 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.c

In [17]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import Boxes, Instances
from detectron2.data.datasets import register_coco_instances
from detectron2.data import build_detection_test_loader

from loguru import logger
import numpy as np
import json
import cv2
import sys

# 설정을 로드
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = ("models/faster_rcnn_R_50_FPN_3x//faster_rcnn_r_50_final_model.pth")  # 훈련된 모델의 가중치 파일 경로
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # 예측에 사용할 임계값
cfg.DATASETS.TEST = ("face_data_set_valid", )  # 테스트 데이터셋 이름
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

# 예측기를 설정
predictor = DefaultPredictor(cfg)

# 예측할 이미지를 로드
image_path = "test.png"  # 예측에 사용할 이미지 경로
image = cv2.imread(image_path)

# 예측을 수행
outputs = predictor(image)

# 예측 결과를 확인
print(outputs)

# 결과 시각화
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
result_image = v.get_image()[:, :, ::-1]

# 결과 이미지를 보기
cv2.imshow("Predictions", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 결과 이미지를 저장
cv2.imwrite("predicted_image.jpg", result_image)


/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=

{'instances': Instances(num_instances=1, image_height=700, image_width=1400, fields=[pred_boxes: Boxes(tensor([[ 755.2027,  120.8748, 1001.8848,  434.5255]], device='cuda:0')), scores: tensor([0.9788], device='cuda:0'), pred_classes: tensor([2], device='cuda:0')])}


True